# Used Car Price Prediction


## Objective

The objective of this competition is to **predict the price of used cars** based on various attributes, such as brand, model, manufacturing year, mileage, <br>
fuel type, engine specifications, and more. Participants are required to build a machine learning model that accurately estimates car prices using the <br>
provided dataset, which contains information about each vehicle's features and characteristics. The model's performance will be evaluated based on <br>
the **Root Mean Squared Error (RMSE)** between the predicted prices and the actual prices.

## Dataset

- **Brand & Model**: Identify the brand or company name along with the specific model of each vehicle.
- **Model Year**: Discover the manufacturing year of the vehicles, crucial for assessing depreciation and technology advancements.
- **Mileage**: Obtain the mileage of each vehicle, a key indicator of wear and tear and potential maintenance requirements.
- **Fuel Type**: Learn about the type of fuel the vehicles run on, whether it's gasoline, diesel, electric, or hybrid.
- **Engine Type**: Understand the engine specifications, shedding light on performance and efficiency.
- **Transmission**: Determine the transmission type, whether automatic, manual, or another variant.
- **Exterior & Interior Colors**: Explore the aesthetic aspects of the vehicles, including exterior and interior color options.
- **Accident History**: Discover whether a vehicle has a prior history of accidents or damage, crucial for informed decision-making.
- **Clean Title**: Evaluate the availability of a clean title, which can impact the vehicle's resale value and legal status.
- **Price**: Access the listed prices for each vehicle, aiding in price comparison and budgeting.

## Initial Data Exploration

### What the data looks like

In [26]:
# Load necessary libraries
import pandas as pd
import numpy as np
import warnings
import re

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.svm import SVR

import category_encoders as ce
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

warnings.filterwarnings("ignore")

In [27]:
# Load the dataset
train_df = pd.read_csv(r'C:\Users\shudi\OneDrive\Desktop\Projects\Kaggle Competition\used_car_price_prediction\train.csv')
test_df = pd.read_csv(r'C:\Users\shudi\OneDrive\Desktop\Projects\Kaggle Competition\used_car_price_prediction\test.csv')

In [28]:
def calculate_missing_percentage(train_df): 
    # Calculate the percentage of missing values
    missing_values_percentage = ((train_df.isnull().sum() / len(train_df)) * 100).round(2)

    # Get data types of each column
    data_types = train_df.dtypes

    # Combine the missing values and data types into a DataFrame for easier viewing
    missing_and_dtypes = pd.DataFrame({
        'Missing Percentage': missing_values_percentage,
        'Data Type': data_types
    })

    print("\nMissing Percentage and Data Types for Each Column:")
    print(missing_and_dtypes)

## Feature Engineering V2


In [49]:
train_df

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horsepower,cylinders,engine_size
0,0,MINI,Cooper S Base,2007,213000,Gasoline Fuel,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,172.0,4,1.6
1,1,Lincoln,LS V8,2002,143250,Gasoline Fuel,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,252.0,8,3.9
2,2,Chevrolet,Silverado 2500 LT,2002,136731,Flex Fuel Capability,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,320.0,8,5.3
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline Fuel,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,420.0,8,5.0
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline Fuel,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,208.0,4,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline Fuel,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500,420.0,8,6.2
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline Fuel,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000,385.0,6,3.0
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline Fuel,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900,469.0,8,4.0
188531,188531,Audi,S5 3.0T Prestige,2022,13895,NaN,3.0L,1-Speed Automatic,Daytona Gray Pearl Effect,Black,None reported,Unknown,84900,NaN,,3.0


### Missing Value Handeling

In [50]:
(train_df == 'not supported').sum()

id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
price           0
horsepower      0
cylinders       0
engine_size     0
dtype: int64

In [51]:
############################
# Missing Value Handeling #
##########################

# Replace "–" and "not supported" with np.nan
train_df.replace("–", np.nan, inplace=True)
train_df.replace("not supported", np.nan, inplace=True)

# Fill missing values in 'clean_title' with "Unknown"
train_df['clean_title'].fillna('Unknown', inplace=True)

In [52]:
train_df.fuel_type.unique()

array(['Gasoline Fuel', 'Flex Fuel Capability', nan,
       'Electric Fuel System', 'Diesel Fuel', 'Plug-In Electric/Gas'],
      dtype=object)

In [53]:
# Use a loop to print each unique value, or convert to a list and print
for value in train_df.engine.unique():
    print(value)

172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel
252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel
320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capability
420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel
208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel
252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel
333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel
355.0HP 5.3L 8 Cylinder Engine Flex Fuel Capability
2.7L V6 24V PDI DOHC Twin Turbo
425.0HP 3.0L Straight 6 Cylinder Engine Gasoline Fuel
312.0HP 3.6L V6 Cylinder Engine Gasoline Fuel
Electric Motor Electric Fuel System
420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel
523.0HP 4.4L 8 Cylinder Engine Gasoline Fuel
503.0HP 4.0L 8 Cylinder Engine Gasoline Fuel
395.0HP 3.0L Straight 6 Cylinder Engine Gasoline/Mild Electric Hybrid
310.0HP 4.6L 8 Cylinder Engine Gasoline Fuel
469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel
5.2L V10 40V PDI DOHC
429.0HP 5.0L 8 Cylinder Engine Gasoline Fuel
210.0HP 3.7L V6 Cylinder Engine Gasoline Fuel
335.0HP 3.0L V6 Cylinder Engine Gasoline/Mild Ele

In [55]:
# Extracting Horsepower
train_df['horsepower'] = train_df["engine"].str.extract(r"(\d+\.\d+)HP", expand=False)

# Extracting Cylinders
train_df['cylinders'] = train_df["engine"].str.extract(r"(\d+) Cylinder|V(\d+)", expand=False).fillna('').sum(axis=1)

# Extracting Engine Size
train_df['engine_size'] = train_df["engine"].str.extract(r"(\d+\.\d+)L", expand=False)

# Extracting Fuel Type
train_df['fuel_type'] = train_df["engine"].str.extract(r"(Gasoline Fuel|Diesel Fuel|Flex Fuel Capability|Electric Fuel System|Plug-In Electric/Gas)", expand=False)


In [61]:
train_df[train_df.cylinders==""].engine.unique()

array(['Electric Motor Electric Fuel System',
       '518.0HP Electric Motor Electric Fuel System',
       '835.0HP Electric Motor Electric Fuel System',
       'Dual Motor - Standard', '2.0L I4 16V PDI DOHC Turbo',
       '2.4L I4 16V GDI DOHC', '2.0 Liter DOHC Turbo',
       '2.0L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo',
       '670.0HP Electric Motor Electric Fuel System', 'Electric',
       '425.0HP Electric Motor Electric Fuel System', '2.0 Liter',
       '3.8L H6 24V GDI DOHC Twin Turbo', '2.0 Liter Turbo',
       '455.0HP Electric Motor Electric Fuel System', '3.0 Liter Turbo',
       '2.0 Liter TFSI', '3.5 Liter', '2.5L H4 16V MPFI DOHC',
       '5.6 Liter', '2.5L I4 16V GDI DOHC', '2.0L I4 16V GDI SOHC Turbo',
       '1.5L I3 12V PDI DOHC Turbo', '5.0 Liter',
       '557.0HP Electric Motor Electric Fuel System',
       '4.0L H6 24V GDI DOHC',
       '200.0HP Electric Motor Electric Fuel System',
       '6.7L I6 24V DDI OHV Turbo Diesel', '3.4L H6 24V MPFI DOHC', n

In [46]:
train_df[train_df.isna().any(axis=1)]

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horsepower,cylinders,engine_size
5,5,Audi,A6 2.0T Sport,2018,40950,Gasoline Fuel,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,White,NaN,None reported,Yes,29950,252.0,4,2.0
8,8,Ford,F-150 XLT,2020,38352,NaN,2.7L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,Snowflake White Pearl Metallic,Black,None reported,Yes,62890,NaN,6,2.7
11,11,Tesla,Model S P100D,2015,81500,Electric Fuel System,Electric Motor Electric Fuel System,1-Speed A/T,Gray,White,None reported,Yes,19000,NaN,,NaN
17,17,Land,Rover Defender SE,2021,46100,NaN,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Gray,Black,None reported,Yes,55000,395.0,6,3.0
20,20,Land,Rover Defender S,2023,7978,NaN,5.2L V10 40V PDI DOHC,8-Speed Automatic,Santorini Black Metallic,Black,None reported,Unknown,41998,NaN,10,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188511,188511,Land,Rover Range Rover Velar P380 SE R-Dynamic,2022,15198,NaN,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Firenze Red,Ebony,None reported,Unknown,23739,NaN,,2.0
188513,188513,Lexus,GS 350 F Sport,2022,82046,NaN,3.5L V6 24V PDI DOHC,8-Speed Automatic,Caviar,Black,None reported,Unknown,48990,NaN,6,3.5
188515,188515,GMC,Sierra 1500 Denali Ultimate,2023,9978,NaN,6.2L V8 16V GDI OHV,10-Speed A/T,White Frost Tri-Coat,Walnut,At least 1 accident or damage reported,Yes,48999,NaN,8,6.2
188520,188520,Mercedes-Benz,GLC 300 GLC 300,2022,18031,NaN,2.0 Liter Turbo,Automatic,White,Black,None reported,Unknown,39998,NaN,,NaN


In [47]:
# Calculate the percentage of missing values for each column
missing_percentage = train_df.isna().mean() * 100

# Display the percentage of missing values for each column
print(missing_percentage)

id               0.000000
brand            0.000000
model            0.000000
model_year       0.000000
milage           0.000000
fuel_type       20.169944
engine           0.490630
transmission     0.046146
ext_col          0.194130
int_col          2.401171
accident         1.300568
clean_title      0.000000
price            0.000000
horsepower      17.640943
cylinders        0.000000
engine_size      7.539264
dtype: float64


### Categorize Numerical Feature 

In [75]:
# Define custom bins with infinity as the upper bound
bins = [0, 30000, 70000, 150000, float('inf')]  # Use float('inf') for very large mileage values
labels = ['Low Mileage', 'Medium Mileage', 'High Mileage', 'Very High Mileage']

# Apply the custom binning
train_df['milage'] = pd.cut(train_df['milage'], bins=bins, labels=labels)

# Check the distribution of the new binned feature
print(train_df['milage'].value_counts())

milage
High Mileage         62343
Low Mileage          57964
Medium Mileage       54829
Very High Mileage    13397
Name: count, dtype: int64


### Categorical Feature Grouping

**brand&model**
1. Combine brand and model into a single feature, as this approach can help capture more granular relationships, then group them based on price level using domain knowledge or a statistical approach like Target Encoding.
2. Target Encoding got picked: Quantile-based binning is recommended due to the skewed distribution, as it ensures each bin has a similar number of observations.


In [76]:
# Combine brand and model into a single feature
train_df['brand_model'] = train_df['brand'] + ' ' + train_df['model']

print(train_df.brand_model.value_counts())
# Apply Target Encoding to the 'model' feature
brand_model_target_encoder = ce.TargetEncoder(cols=['brand_model'])
train_df['brand_model_level'] = brand_model_target_encoder.fit_transform(train_df['brand_model'], train_df['price'])
print("")
print(train_df.brand_model_level.describe())

brand_model
Ford F-150 XLT                          2934
BMW M3 Base                             2195
Chevrolet Camaro 2SS                    1699
BMW M4 Base                             1608
Ford Mustang GT Premium                 1515
                                        ... 
Volkswagen Veyron 16.4 Grand Sport         1
McLaren R8 5.2 V10 performance             1
Chevrolet Wrangler Unlimited Rubicon       1
Toyota F-250 XLT SuperCab Super Duty       1
Ford Model X P100D                         1
Name: count, Length: 3555, dtype: int64

count    188533.000000
mean      44002.228401
std       25273.243867
min        9615.515748
25%       27174.630096
50%       38830.952522
75%       55295.017744
max      292010.097374
Name: brand_model_level, dtype: float64


In [77]:
# Group the brand_model_encoded feature into 5 price levels using quantiles
train_df['brand_model_level'] = pd.qcut(train_df['brand_model_level'], q=5, labels=['Low', 'Lower-Mid', 'Mid', 'Upper-Mid', 'High'])

# Check the distribution of the new price level feature
print(train_df['brand_model_level'].value_counts())

brand_model_level
Mid          37828
Low          37718
High         37706
Lower-Mid    37705
Upper-Mid    37576
Name: count, dtype: int64


**model_year**

Since the distribution is heavily skewed towards more recent years, a custom binning based on both car age and vehicle trends (such as technological advancements) would make the most sense. Here’s a potential grouping strategy:

Custom Grouping Based on Recent Years and Technology:

1. New Cars (2020 and newer):

	•	These cars are generally very new, and they are likely to have the latest technology and features. They will also have the least depreciation.
	•	2020, 2021, 2022, 2023, 2024

2. Recently Used Cars (2015-2019):

	•	Cars from the last 5 to 10 years often still have many modern features, but they have seen some depreciation.
	•	2015-2019

3. Older Modern Cars (2010-2014):

	•	These cars are older but still represent relatively modern features (many improvements in fuel efficiency, safety, and infotainment systems).
	•	2010-2014

4. Mid-2000s Cars (2000-2009):

	•	These cars are over a decade old, with significant depreciation. They likely have dated technology but are still common on the market.
	•	2000-2009

5. Vintage Cars (Before 2000):

	•	Cars older than 2000 are now considered vintage or heavily used, with minimal resale value or targeted at collectors.
	•	Before 2000

Final Grouping Strategy:

	•	2020 and newer: Modern Cars
	•	2015-2019: Recently Used Cars
	•	2010-2014: Older Modern Cars
	•	2000-2009: Mid-2000s Cars
	•	Before 2000: Vintage Cars
    

In [78]:
# Define custom bin ranges based on model year distribution
bins = [0, 1999, 2009, 2014, 2019, float('inf')]  # Group by periods
labels = ['Vintage Cars', 'Mid-2000s Cars', 'Older Modern Cars', 'Recently Used Cars', 'Modern Cars']

# Apply the custom binning to the 'model_year' column
train_df['model_year'] = pd.cut(train_df['model_year'], bins=bins, labels=labels)

# Check the distribution of the new year groups
print(train_df['model_year'].value_counts())

model_year
Recently Used Cars    69702
Modern Cars           58726
Older Modern Cars     31806
Mid-2000s Cars        26541
Vintage Cars           1758
Name: count, dtype: int64


**fuel_type**

	1.	Gasoline: This is the most common and should remain a separate category because its price characteristics differ from other fuel types.
	2.	Diesel: Keep this as a separate group because diesel vehicles, particularly trucks and large cars, often have different pricing dynamics compared to gasoline cars.
	3.	Hybrid and Plug-In Hybrid: These vehicles can be grouped together under “Hybrid”, as both are seen as more fuel-efficient and eco-friendly alternatives to gasoline, affecting the price similarly.
	4.	Electric (EV): These should be grouped separately because electric vehicles typically have a higher upfront cost due to battery technology, and they are rapidly gaining popularity.
	5.	Other/Unknown: Handle categories like Flex Fuel (E85), Not Supported, and missing values (”–”) by  grouping them as “Other” or assigning them to a default category like gasoline if appropriate.


In [79]:
# Define custom fuel type groups based on domain knowledge
def group_fuel_type(fuel):
    if fuel == 'Gasoline':
        return 'Gasoline'
    elif fuel == 'Diesel':
        return 'Diesel'
    elif fuel in ['Hybrid', 'Plug-In Hybrid']:
        return 'Hybrid'
    elif fuel == 'Electric':
        return 'Electric'
    else:
        return 'Other'  # For Flex Fuel, unknowns, and missing values

# Apply the grouping
train_df['fuel_type'] = train_df['fuel_type'].apply(group_fuel_type)

# Check the distribution of the grouped fuel types
print(train_df['fuel_type'].value_counts())

fuel_type
Gasoline    171023
Hybrid        7353
Other         6202
Diesel        3955
Name: count, dtype: int64


**transmission**

	1.	Basic Automatic (A/T): This includes most of the standard automatic transmissions, such as 4-speed and 6-speed automatics. These are common and priced in the mid-range.
	2.	Advanced Automatic (8-speed, 9-speed, 10-speed, etc.): These offer more gears and better fuel efficiency, often found in more expensive or newer cars.
	3.	Manual Transmission (M/T): Manuals are generally less expensive, except for certain sports cars, where they are favored by enthusiasts.
	4.	CVT: CVTs offer better fuel efficiency but tend to be priced similarly to standard automatics, let label it as Basic Automatic since the volumn is very small.
	5.	Dual-Clutch / Semi-Automatic (DCT): These are advanced, performance-oriented transmissions commonly found in sports and luxury cars.
	6.	Other / Unknown: Treat any unknown or rare types of transmission as Basic Automatic.


In [80]:
# Define custom transmission groups based on domain knowledge
def group_transmission(transmission):
    if 'A/T' in transmission or 'Automatic' in transmission:
        if '8-Speed' in transmission or '9-Speed' in transmission or '10-Speed' in transmission:
            return 'Advanced Automatic'
        else:
            return 'Basic Automatic'
    elif 'M/T' in transmission or 'Manual' in transmission:
        return 'Manual'
    elif 'CVT' in transmission:
        return 'Basic Automatic'
    elif 'Dual' in transmission or 'DCT' in transmission or 'Semi-Automatic' in transmission:
        return 'Dual-Clutch'
    else:
        return 'Basic Automatic'  # For any unknown or unsupported transmission types

# Apply the grouping
train_df['transmission'] = train_df['transmission'].apply(group_transmission)

# Check the distribution of the grouped transmission types
print(train_df['transmission'].value_counts())

transmission
Basic Automatic       105823
Advanced Automatic     47097
Dual-Clutch            19255
Manual                 16358
Name: count, dtype: int64


**engine**

Extract three columns from the engine column:


***Cylinder**

The number of cylinders is closely linked to both the performance and the price of a vehicle. Vehicles with more cylinders tend to have higher power output but are also less fuel-efficient.

Suggested Grouping (based on domain knowledge):

	1.	3-Cylinder Engines:
	•	Typically found in small, ultra-economical cars. This is the least common configuration.
	2.	4-Cylinder Engines:
	•	Found in economy cars, compact cars, and some mid-range sedans. These engines are popular for their fuel efficiency.
	3.	6-Cylinder Engines:
	•	Mid-range engines that provide a balance between power and efficiency. Common in mid-size sedans, SUVs, and performance cars.
	4.	8-Cylinder Engines and Above:
	•	High-performance engines found in sports cars, trucks, and luxury vehicles. These engines deliver the highest power output and are associated with premium vehicles.


**Horsepower**

Horsepower is a key indicator of vehicle performance and has a significant impact on price. The data shows a broad range of horsepower values, from 70 HP to 1020 HP. Here’s how we might group horsepower into meaningful categories:

Suggested Grouping (based on data and domain knowledge):

	1.	Low Horsepower (<200 HP):
	•	Typically found in economy and compact cars. These vehicles are usually cheaper and designed for fuel efficiency rather than performance.
	2.	Moderate Horsepower (200-300 HP):
	•	Common in mid-sized sedans, crossovers, and SUVs. Vehicles in this range offer a balance of performance and efficiency.
	3.	High Horsepower (300-450 HP):
	•	Often found in performance-oriented vehicles, luxury cars, and trucks. Cars in this range start to offer significantly higher performance and, consequently, higher prices.
	4.	Very High Horsepower (>450 HP):
	•	Sports cars, luxury SUVs, and high-performance vehicles. This category includes premium and expensive vehicles, such as high-end sports cars.

**Engine Size**

Engine size, or displacement, affects both performance and fuel efficiency. Larger engines tend to offer more power but are less fuel-efficient. The data shows engine sizes ranging from 0.65L to 8.4L.

Suggested Grouping (based on data and domain knowledge):

	1.	Small Engines (<2.0L):
	•	Found in economy cars, compact cars, and hybrids. These vehicles are focused on fuel efficiency rather than performance.
	2.	Moderate Engines (2.0L-3.5L):
	•	Common in mid-size sedans and SUVs. These engines offer a balance between power and fuel efficiency, often found in popular consumer vehicles.
	3.	Large Engines (3.5L-5.0L):
	•	Found in performance cars, trucks, and larger SUVs. These engines offer more power but are less fuel-efficient.
	4.	Very Large Engines (>5.0L):
	•	High-performance, luxury, or specialty vehicles, including sports cars and trucks. These are typically the most expensive vehicles due to their power.


Impute missing values with the mode.

In [27]:
# Define a function to extract horsepower, engine size, and cylinders
def extract_engine_features(engine_str):
    # Extract horsepower (e.g., 252.0HP -> 252)
    hp_match = re.search(r'(\d+)\.0HP', engine_str)
    horsepower = int(hp_match.group(1)) if hp_match else None
    
    # Extract engine size (e.g., 1.6L -> 1.6)
    engine_size_match = re.search(r'(\d+\.\d+)L', engine_str)
    engine_size = float(engine_size_match.group(1)) if engine_size_match else None
    
    # Extract cylinders (e.g., 4 Cylinder -> 4)
    cylinder_match = re.search(r'(\d+) Cylinder', engine_str)
    cylinders = int(cylinder_match.group(1)) if cylinder_match else None
    
    return horsepower, engine_size, cylinders


In [28]:
# Apply the function to extract the three features into new columns
train_df[['horsepower', 'engine_size', 'cylinders']] = train_df['engine'].apply(lambda x: pd.Series(extract_engine_features(x)))

In [29]:
calculate_missing_percentage(train_df[['horsepower', 'engine_size', 'cylinders']])


Missing Percentage and Data Types for Each Column:
             Missing Percentage Data Type
horsepower                17.64   float64
engine_size                7.54   float64
cylinders                 20.08   float64


In [30]:
train_df[['horsepower', 'engine_size', 'cylinders']].describe()

,horsepower,engine_size,cylinders
count,155274.000000,174319.000000,150678.000000
mean,343.263856,3.806676,6.374268
std,114.734210,1.355945,1.554579
min,70.000000,0.650000,3.000000
25%,263.000000,3.000000,6.000000
50%,329.000000,3.500000,6.000000
75%,416.000000,4.800000,8.000000
max,1020.000000,8.400000,12.000000


In [31]:
# Create custom bins for horsepower, engine size, and cylinders, including an "Unknown" label for missing values

# 1. Horsepower Grouping with 'Unknown'
hp_bins = [0, 200, 300, 450, float('inf')]
hp_labels = ['Low HP', 'Moderate HP', 'High HP', 'Very High HP', 'Unknown']
train_df['horsepower'] = pd.cut(train_df['horsepower'], bins=hp_bins, labels=hp_labels[:-1], include_lowest=True)
train_df['horsepower'] = train_df['horsepower'].cat.add_categories('Unknown').fillna('Unknown')
print(train_df['horsepower'].value_counts())

# 2. Engine Size Grouping with 'Unknown'
engine_size_bins = [0, 2.0, 3.5, 5.0, float('inf')]
engine_size_labels = ['Small', 'Moderate', 'Large', 'Very Large', 'Unknown']
train_df['engine_size'] = pd.cut(train_df['engine_size'], bins=engine_size_bins, labels=engine_size_labels[:-1], include_lowest=True)
train_df['engine_size'] = train_df['engine_size'].cat.add_categories('Unknown').fillna('Unknown')
print(train_df['engine_size'].value_counts())

# 3. Cylinders Grouping with 'Unknown'
cylinder_bins = [0, 4, 6, float('inf')]
cylinder_labels = ['4 Cylinder', '6 Cylinder', '8 Cylinder or more', 'Unknown']
train_df['cylinders'] = pd.cut(train_df['cylinders'], bins=cylinder_bins, labels=cylinder_labels[:-1], include_lowest=True)
train_df['cylinders'] = train_df['cylinders'].cat.add_categories('Unknown').fillna('Unknown')
print(train_df['cylinders'].value_counts())

horsepower
High HP         69129
Moderate HP     48334
Unknown         33259
Very High HP    24114
Low HP          13697
Name: count, dtype: int64
engine_size
Moderate      64164
Large         48541
Very Large    36899
Small         24715
Unknown       14214
Name: count, dtype: int64
cylinders
6 Cylinder            66928
8 Cylinder or more    54552
Unknown               37855
4 Cylinder            29198
Name: count, dtype: int64


**ext_col**

Explanation:

	1.	PopularColor: This remains the largest group and includes the most neutral and widely accepted car colors.
	2.	SportyColor: This group includes bright and performance-focused colors such as red, yellow, and orange.
	3.	OtherColor: Now includes all remaining colors, including luxury metallics and pearls, as well as rare and niche colors.


In [32]:
# Adjusted function to merge LuxuryColor into OtherColor and simplify groups
def group_ext_color(color):
    if color in ['Black', 'White', 'Silver', 'Gray', 'Ebony Black', 'Oxford White', 'Platinum White Pearl']:
        return 'PopularColors'
    elif color in ['Red', 'Yellow', 'Orange', 'Firecracker Red Clearcoat', 'Rosso Mars Metallic', 'Lizard Green', 'Python Green', 'Bright White']:
        return 'SportyColors'
    else:
        return 'OtherColors'  # Merge LuxuryColor and any remaining colors into OtherColor

# Apply the final grouping function to the ext_color column
train_df['ext_col'] = train_df['ext_col'].apply(group_ext_color)

# Check the final distribution of the grouped exterior colors
print(train_df['ext_col'].value_counts())

ext_col
PopularColors    135276
OtherColors       40700
SportyColors      12557
Name: count, dtype: int64


**int_col**

	•	CommonColors: At over 158,000 entries, this group reflects the market’s preference for neutral, practical interior colors that don’t significantly impact car pricing. This group likely has stable, predictable pricing effects.
	•	LuxuryColors: At 7,240 entries, this group is now large enough to represent premium and niche interiors, which typically increase a car’s value. This size makes it meaningful in the model, without being too small to overfit.
	•	SportyColors: At 7,318 entries, this group remains relevant for performance or custom cars, where bold, unique colors are important. This group has the potential to highlight pricing effects for sports and performance vehicles.
	•	UnknownColors: At 15,590 entries, this group appropriately handles missing or unspecified values, allowing the model to capture the possible effect of unknown interior colors on pricing.

In [33]:
# Adjust the function to merge UncommonColors into LuxuryColors
def group_int_color(color):
    # Neutral / Common Colors
    if color in ['Black', 'Gray', 'Beige', 'Ebony', 'Jet Black', 'Slate', 'Light Gray', 'Medium Earth Gray', 
                 'Medium Ash Gray', 'Dark Ash', 'Charcoal', 'Graphite', 'Ebony Black']:
        return 'CommonColors'
    
    # Luxury / Premium Colors (including Uncommon Colors)
    elif color in ['Tan', 'Brown', 'Ivory', 'Cappuccino', 'Camel', 'Brandy', 'Parchment', 'Canberra Beige', 
                   'Oyster', 'Saddle', 'Macchiato', 'Ebony / Ebony Accents', 'Sahara Tan', 'Roast', 'Almond Beige', 
                   'Green', 'Purple', 'Gold', 'Boulder', 'Deep Chestnut', 'Tupelo', 'Gideon', 'Hotspur', 'Chestnut', 
                   'Dark Auburn', 'Porpoise', 'Deep Garnet']:  # UncommonColors merged here
        return 'LuxuryColors'
    
    # Sporty / Unique Colors
    elif color in ['Red', 'Blue', 'Orange', 'Adrenaline Red', 'Sakhir Orange', 'Rioja Red', 'Pimento Red w/Ebony', 
                   'Pimento / Ebony', 'Black w/Red Stitching', 'Red / Black', 'Cobalt Blue', 'Kyalami Orange']:
        return 'SportyColors'
    
    # Handle missing or unspecified colors
    else:
        return 'UnknownColors'

# Apply the adjusted grouping function to the int_color column
train_df['int_col'] = train_df['int_col'].apply(group_int_color)

# Check the final distribution of the grouped interior colors
print(train_df['int_col'].value_counts())

int_col
CommonColors     158385
UnknownColors     15590
SportyColors       7318
LuxuryColors       7240
Name: count, dtype: int64


**accident**

Convert it to boolean column

In [34]:
# Convert accident values into a binary flag (0/1)
train_df['accident'] = train_df['accident'].apply(lambda x: 1 if x == 'At least 1 accident or damage reported' else 0)

# Check the distribution of the binary flag
print(train_df['accident'].value_counts())

accident
0    146966
1     41567
Name: count, dtype: int64


**clean_title**

In [35]:
# Convert accident values into a binary flag (0/1)
train_df['clean_title'] = train_df['clean_title'].apply(lambda x: 1 if x == 'Yes' else 0)

# Check the distribution of the binary flag
print(train_df['clean_title'].value_counts())

clean_title
1    167114
0     21419
Name: count, dtype: int64


In [36]:
final_columns = ["brand_model_level", "model_year", "milage", "fuel_type", "transmission", "ext_col", "int_col", "accident", "clean_title", "horsepower", "engine_size", "cylinders", "price"]

In [37]:
train_df

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,brand_model,brand_model_level,horsepower,engine_size,cylinders
0,0,MINI,Cooper S Base,Mid-2000s Cars,Very High Mileage,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,Basic Automatic,SportyColors,CommonColors,0,1,4200,MINI Cooper S Base,Low,Low HP,Small,4 Cylinder
1,1,Lincoln,LS V8,Mid-2000s Cars,High Mileage,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,Basic Automatic,PopularColors,CommonColors,1,1,4999,Lincoln LS V8,Low,Moderate HP,Large,8 Cylinder or more
2,2,Chevrolet,Silverado 2500 LT,Mid-2000s Cars,High Mileage,Other,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,Basic Automatic,OtherColors,CommonColors,0,1,13900,Chevrolet Silverado 2500 LT,Lower-Mid,High HP,Very Large,8 Cylinder or more
3,3,Genesis,G90 5.0 Ultimate,Recently Used Cars,Low Mileage,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Dual-Clutch,PopularColors,CommonColors,0,1,45000,Genesis G90 5.0 Ultimate,Upper-Mid,High HP,Large,8 Cylinder or more
4,4,Mercedes-Benz,Metris Base,Modern Cars,Low Mileage,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,Basic Automatic,PopularColors,CommonColors,0,1,97500,Mercedes-Benz Metris Base,High,Moderate HP,Small,4 Cylinder
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Cadillac,Escalade ESV Platinum,Recently Used Cars,Medium Mileage,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Dual-Clutch,PopularColors,CommonColors,0,1,27500,Cadillac Escalade ESV Platinum,Upper-Mid,High HP,Very Large,8 Cylinder or more
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,Recently Used Cars,Low Mileage,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,Advanced Automatic,PopularColors,CommonColors,1,1,30000,Mercedes-Benz AMG C 43 AMG C 43 4MATIC,Upper-Mid,High HP,Moderate,6 Cylinder
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,Modern Cars,Low Mileage,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,Basic Automatic,PopularColors,CommonColors,0,1,86900,Mercedes-Benz AMG GLC 63 Base 4MATIC,High,Very High HP,Large,8 Cylinder or more
188531,188531,Audi,S5 3.0T Prestige,Modern Cars,Low Mileage,Gasoline,3.0L,Basic Automatic,OtherColors,CommonColors,0,0,84900,Audi S5 3.0T Prestige,Mid,Unknown,Moderate,Unknown


In [38]:
train_df_final = train_df[final_columns]

In [39]:
train_df_final

,brand_model_level,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,horsepower,engine_size,cylinders,price
0,Low,Mid-2000s Cars,Very High Mileage,Gasoline,Basic Automatic,SportyColors,CommonColors,0,1,Low HP,Small,4 Cylinder,4200
1,Low,Mid-2000s Cars,High Mileage,Gasoline,Basic Automatic,PopularColors,CommonColors,1,1,Moderate HP,Large,8 Cylinder or more,4999
2,Lower-Mid,Mid-2000s Cars,High Mileage,Other,Basic Automatic,OtherColors,CommonColors,0,1,High HP,Very Large,8 Cylinder or more,13900
3,Upper-Mid,Recently Used Cars,Low Mileage,Gasoline,Dual-Clutch,PopularColors,CommonColors,0,1,High HP,Large,8 Cylinder or more,45000
4,High,Modern Cars,Low Mileage,Gasoline,Basic Automatic,PopularColors,CommonColors,0,1,Moderate HP,Small,4 Cylinder,97500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,Upper-Mid,Recently Used Cars,Medium Mileage,Gasoline,Dual-Clutch,PopularColors,CommonColors,0,1,High HP,Very Large,8 Cylinder or more,27500
188529,Upper-Mid,Recently Used Cars,Low Mileage,Gasoline,Advanced Automatic,PopularColors,CommonColors,1,1,High HP,Moderate,6 Cylinder,30000
188530,High,Modern Cars,Low Mileage,Gasoline,Basic Automatic,PopularColors,CommonColors,0,1,Very High HP,Large,8 Cylinder or more,86900
188531,Mid,Modern Cars,Low Mileage,Gasoline,Basic Automatic,OtherColors,CommonColors,0,0,Unknown,Moderate,Unknown,84900


## Apply Feature Engineering to Testset

In [40]:
# Define custom fuel type groups based on domain knowledge
def group_fuel_type(fuel):
    if fuel == 'Gasoline':
        return 'Gasoline'
    elif fuel == 'Diesel':
        return 'Diesel'
    elif fuel in ['Hybrid', 'Plug-In Hybrid']:
        return 'Hybrid'
    elif fuel == 'Electric':
        return 'Electric'
    else:
        return 'Other'  # For Flex Fuel, unknowns, and missing values

# Define custom transmission groups based on domain knowledge
def group_transmission(transmission):
    if 'A/T' in transmission or 'Automatic' in transmission:
        if '8-Speed' in transmission or '9-Speed' in transmission or '10-Speed' in transmission:
            return 'Advanced Automatic'
        else:
            return 'Basic Automatic'
    elif 'M/T' in transmission or 'Manual' in transmission:
        return 'Manual'
    elif 'CVT' in transmission:
        return 'Basic Automatic'
    elif 'Dual' in transmission or 'DCT' in transmission or 'Semi-Automatic' in transmission:
        return 'Dual-Clutch'
    else:
        return 'Basic Automatic'  # For any unknown or unsupported transmission types
    
# Define a function to extract horsepower, engine size, and cylinders
def extract_engine_features(engine_str):
    # Extract horsepower (e.g., 252.0HP -> 252)
    hp_match = re.search(r'(\d+)\.0HP', engine_str)
    horsepower = int(hp_match.group(1)) if hp_match else None
    
    # Extract engine size (e.g., 1.6L -> 1.6)
    engine_size_match = re.search(r'(\d+\.\d+)L', engine_str)
    engine_size = float(engine_size_match.group(1)) if engine_size_match else None
    
    # Extract cylinders (e.g., 4 Cylinder -> 4)
    cylinder_match = re.search(r'(\d+) Cylinder', engine_str)
    cylinders = int(cylinder_match.group(1)) if cylinder_match else None
    
    return horsepower, engine_size, cylinders

# Adjusted function to merge LuxuryColor into OtherColor and simplify groups
def group_ext_color(color):
    if color in ['Black', 'White', 'Silver', 'Gray', 'Ebony Black', 'Oxford White', 'Platinum White Pearl']:
        return 'PopularColors'
    elif color in ['Red', 'Yellow', 'Orange', 'Firecracker Red Clearcoat', 'Rosso Mars Metallic', 'Lizard Green', 'Python Green', 'Bright White']:
        return 'SportyColors'
    else:
        return 'OtherColors'  # Merge LuxuryColor and any remaining colors into OtherColor


# Adjust the function to merge UncommonColors into LuxuryColors
def group_int_color(color):
    # Neutral / Common Colors
    if color in ['Black', 'Gray', 'Beige', 'Ebony', 'Jet Black', 'Slate', 'Light Gray', 'Medium Earth Gray', 
                 'Medium Ash Gray', 'Dark Ash', 'Charcoal', 'Graphite', 'Ebony Black']:
        return 'CommonColors'
    
    # Luxury / Premium Colors (including Uncommon Colors)
    elif color in ['Tan', 'Brown', 'Ivory', 'Cappuccino', 'Camel', 'Brandy', 'Parchment', 'Canberra Beige', 
                   'Oyster', 'Saddle', 'Macchiato', 'Ebony / Ebony Accents', 'Sahara Tan', 'Roast', 'Almond Beige', 
                   'Green', 'Purple', 'Gold', 'Boulder', 'Deep Chestnut', 'Tupelo', 'Gideon', 'Hotspur', 'Chestnut', 
                   'Dark Auburn', 'Porpoise', 'Deep Garnet']:  # UncommonColors merged here
        return 'LuxuryColors'
    
    # Sporty / Unique Colors
    elif color in ['Red', 'Blue', 'Orange', 'Adrenaline Red', 'Sakhir Orange', 'Rioja Red', 'Pimento Red w/Ebony', 
                   'Pimento / Ebony', 'Black w/Red Stitching', 'Red / Black', 'Cobalt Blue', 'Kyalami Orange']:
        return 'SportyColors'
    
    # Handle missing or unspecified colors
    else:
        return 'UnknownColors'



In [41]:
def feature_engineering(test_df):
    
    ############################
    # Missing Value Handeling #
    ##########################

    # Step 1: Fill missing values in 'clean_title' with "Unknown"
    test_df['clean_title'].fillna('Unknown', inplace=True)

    for column in test_df.columns:
        if (column != 'clean_title'):  # Exclude 'clean_title' since it's already handled
            test_df[column].fillna(test_df[column].mode()[0], inplace=True)

    # Verify missing values after filling
    print("Missing values after applying the strategy:")
    calculate_missing_percentage(test_df)
    
    # Define custom bins with infinity as the upper bound
    bins = [0, 30000, 70000, 150000, float('inf')]  # Use float('inf') for very large mileage values
    labels = ['Low Mileage', 'Medium Mileage', 'High Mileage', 'Very High Mileage']

    # Apply the custom binning
    test_df['milage'] = pd.cut(test_df['milage'], bins=bins, labels=labels)

    # Check the distribution of the new binned feature
    print(test_df['milage'].value_counts())
    
    # Combine brand and model into a single feature
    test_df['brand_model'] = test_df['brand'] + ' ' + test_df['model']

    print(test_df.brand_model.value_counts())
    # Apply Target Encoding to the 'model' feature
    # brand_model_target_encoder = ce.TargetEncoder(cols=['brand_model'])
    test_df['brand_model_level'] = brand_model_target_encoder.transform(test_df['brand_model'])
    print("")
    print(test_df.brand_model_level.describe())
    
    # Group the brand_model_encoded feature into 5 price levels using quantiles
    test_df['brand_model_level'] = pd.qcut(test_df['brand_model_level'], q=5, labels=['Low', 'Lower-Mid', 'Mid', 'Upper-Mid', 'High'])

    # Check the distribution of the new price level feature
    print(test_df['brand_model_level'].value_counts())

    # Define custom bin ranges based on model year distribution
    bins = [0, 1999, 2009, 2014, 2019, float('inf')]  # Group by periods
    labels = ['Vintage Cars', 'Mid-2000s Cars', 'Older Modern Cars', 'Recently Used Cars', 'Modern Cars']

    # Apply the custom binning to the 'model_year' column
    test_df['model_year'] = pd.cut(test_df['model_year'], bins=bins, labels=labels)

    # Check the distribution of the new year groups
    print(test_df['model_year'].value_counts())

    # Apply the grouping
    test_df['fuel_type'] = test_df['fuel_type'].apply(group_fuel_type)

    # Check the distribution of the grouped fuel types
    print(test_df['fuel_type'].value_counts())

    # Apply the grouping
    test_df['transmission'] = test_df['transmission'].apply(group_transmission)

    # Check the distribution of the grouped transmission types
    print(test_df['transmission'].value_counts())

    # Apply the function to extract the three features into new columns
    test_df[['horsepower', 'engine_size', 'cylinders']] = test_df['engine'].apply(lambda x: pd.Series(extract_engine_features(x)))

    # Create custom bins for horsepower, engine size, and cylinders, including an "Unknown" label for missing values

    # 1. Horsepower Grouping with 'Unknown'
    hp_bins = [0, 200, 300, 450, float('inf')]
    hp_labels = ['Low HP', 'Moderate HP', 'High HP', 'Very High HP', 'Unknown']
    test_df['horsepower'] = pd.cut(test_df['horsepower'], bins=hp_bins, labels=hp_labels[:-1], include_lowest=True)
    test_df['horsepower'] = test_df['horsepower'].cat.add_categories('Unknown').fillna('Unknown')
    print(test_df['horsepower'].value_counts())

    # 2. Engine Size Grouping with 'Unknown'
    engine_size_bins = [0, 2.0, 3.5, 5.0, float('inf')]
    engine_size_labels = ['Small', 'Moderate', 'Large', 'Very Large', 'Unknown']
    test_df['engine_size'] = pd.cut(test_df['engine_size'], bins=engine_size_bins, labels=engine_size_labels[:-1], include_lowest=True)
    test_df['engine_size'] = test_df['engine_size'].cat.add_categories('Unknown').fillna('Unknown')
    print(test_df['engine_size'].value_counts())

    # 3. Cylinders Grouping with 'Unknown'
    cylinder_bins = [0, 4, 6, float('inf')]
    cylinder_labels = ['4 Cylinder', '6 Cylinder', '8 Cylinder or more', 'Unknown']
    test_df['cylinders'] = pd.cut(test_df['cylinders'], bins=cylinder_bins, labels=cylinder_labels[:-1], include_lowest=True)
    test_df['cylinders'] = test_df['cylinders'].cat.add_categories('Unknown').fillna('Unknown')
    print(test_df['cylinders'].value_counts())

    # Apply the final grouping function to the ext_color column
    test_df['ext_col'] = test_df['ext_col'].apply(group_ext_color)

    # Check the final distribution of the grouped exterior colors
    print(test_df['ext_col'].value_counts())

    # Apply the adjusted grouping function to the int_color column
    test_df['int_col'] = test_df['int_col'].apply(group_int_color)

    # Check the final distribution of the grouped interior colors
    print(test_df['int_col'].value_counts())

    # Convert accident values into a binary flag (0/1)
    test_df['accident'] = test_df['accident'].apply(lambda x: 1 if x == 'At least 1 accident or damage reported' else 0)

    # Check the distribution of the binary flag
    print(test_df['accident'].value_counts())

    # Convert accident values into a binary flag (0/1)
    test_df['clean_title'] = test_df['clean_title'].apply(lambda x: 1 if x == 'Yes' else 0)

    # Check the distribution of the binary flag
    print(test_df['clean_title'].value_counts())

    
    return test_df

In [42]:
test_df = feature_engineering(test_df)

Missing values after applying the strategy:

Missing Percentage and Data Types for Each Column:
              Missing Percentage Data Type
id                           0.0     int64
brand                        0.0    object
model                        0.0    object
model_year                   0.0     int64
milage                       0.0     int64
fuel_type                    0.0    object
engine                       0.0    object
transmission                 0.0    object
ext_col                      0.0    object
int_col                      0.0    object
accident                     0.0    object
clean_title                  0.0    object
milage
High Mileage         41477
Low Mileage          38451
Medium Mileage       36610
Very High Mileage     9152
Name: count, dtype: int64
brand_model
Ford F-150 XLT                   1915
BMW M3 Base                      1400
Chevrolet Camaro 2SS             1104
BMW M4 Base                       999
Ford Mustang GT Premium           937
  

In [43]:
final_columns

['brand_model_level',
 'model_year',
 'milage',
 'fuel_type',
 'transmission',
 'ext_col',
 'int_col',
 'accident',
 'clean_title',
 'horsepower',
 'engine_size',
 'cylinders',
 'price']

In [44]:
test_df_final = test_df[['brand_model_level',
 'model_year',
 'milage',
 'fuel_type',
 'transmission',
 'ext_col',
 'int_col',
 'accident',
 'clean_title',
 'horsepower',
 'engine_size',
 'cylinders']]

In [45]:
test_df

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,brand_model,brand_model_level,horsepower,engine_size,cylinders
0,188533,Land,Rover LR2 Base,Recently Used Cars,High Mileage,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,Basic Automatic,PopularColors,CommonColors,0,1,Land Rover LR2 Base,Low,Moderate HP,Small,4 Cylinder
1,188534,Land,Rover Defender SE,Modern Cars,Low Mileage,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Advanced Automatic,PopularColors,CommonColors,0,1,Land Rover Defender SE,High,High HP,Moderate,6 Cylinder
2,188535,Ford,Expedition Limited,Modern Cars,Low Mileage,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,Advanced Automatic,PopularColors,CommonColors,0,0,Ford Expedition Limited,Upper-Mid,Unknown,Moderate,Unknown
3,188536,Audi,A6 2.0T Sport,Recently Used Cars,Medium Mileage,Gasoline,2.0 Liter TFSI,Basic Automatic,OtherColors,CommonColors,0,0,Audi A6 2.0T Sport,Lower-Mid,Unknown,Unknown,Unknown
4,188537,Audi,A6 2.0T Premium Plus,Recently Used Cars,Medium Mileage,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,Basic Automatic,PopularColors,CommonColors,0,1,Audi A6 2.0T Premium Plus,Lower-Mid,Moderate HP,Small,4 Cylinder
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125685,314218,Mercedes-Benz,GL-Class GL 450 4MATIC,Older Modern Cars,High Mileage,Gasoline,362.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,Basic Automatic,PopularColors,CommonColors,0,1,Mercedes-Benz GL-Class GL 450 4MATIC,Mid,High HP,Moderate,6 Cylinder
125686,314219,Audi,Q7 55 Prestige,Recently Used Cars,Low Mileage,Gasoline,3.0 Liter Turbo,Basic Automatic,PopularColors,CommonColors,0,0,Audi Q7 55 Prestige,Upper-Mid,Unknown,Unknown,Unknown
125687,314220,Audi,A6 3.0T Premium Plus,Older Modern Cars,High Mileage,Gasoline,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,Basic Automatic,PopularColors,CommonColors,0,1,Audi A6 3.0T Premium Plus,Lower-Mid,High HP,Moderate,6 Cylinder
125688,314221,Audi,Q7 3.0T Premium,Older Modern Cars,High Mileage,Gasoline,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,Basic Automatic,PopularColors,CommonColors,0,1,Audi Q7 3.0T Premium,Mid,High HP,Moderate,6 Cylinder


In [46]:
test_df_final

,brand_model_level,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,horsepower,engine_size,cylinders
0,Low,Recently Used Cars,High Mileage,Gasoline,Basic Automatic,PopularColors,CommonColors,0,1,Moderate HP,Small,4 Cylinder
1,High,Modern Cars,Low Mileage,Hybrid,Advanced Automatic,PopularColors,CommonColors,0,1,High HP,Moderate,6 Cylinder
2,Upper-Mid,Modern Cars,Low Mileage,Gasoline,Advanced Automatic,PopularColors,CommonColors,0,0,Unknown,Moderate,Unknown
3,Lower-Mid,Recently Used Cars,Medium Mileage,Gasoline,Basic Automatic,OtherColors,CommonColors,0,0,Unknown,Unknown,Unknown
4,Lower-Mid,Recently Used Cars,Medium Mileage,Gasoline,Basic Automatic,PopularColors,CommonColors,0,1,Moderate HP,Small,4 Cylinder
...,...,...,...,...,...,...,...,...,...,...,...,...
125685,Mid,Older Modern Cars,High Mileage,Gasoline,Basic Automatic,PopularColors,CommonColors,0,1,High HP,Moderate,6 Cylinder
125686,Upper-Mid,Recently Used Cars,Low Mileage,Gasoline,Basic Automatic,PopularColors,CommonColors,0,0,Unknown,Unknown,Unknown
125687,Lower-Mid,Older Modern Cars,High Mileage,Gasoline,Basic Automatic,PopularColors,CommonColors,0,1,High HP,Moderate,6 Cylinder
125688,Mid,Older Modern Cars,High Mileage,Gasoline,Basic Automatic,PopularColors,CommonColors,0,1,High HP,Moderate,6 Cylinder


## Model Selection

In [58]:
# Step 1: Define features and target for training
categorical_features = ["brand_model_level", "model_year", "milage", "fuel_type", "transmission", 
                        "ext_col", "int_col", "accident", "clean_title", "horsepower", "engine_size", "cylinders"]
target = "price"

# Step 2: Convert categorical columns to pandas 'category' dtype
for col in categorical_features:
    train_df[col] = train_df[col].astype('category')

# Step 3: Split the dataset into training and testing
X = train_df[categorical_features]
y = train_df[target]

# Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Identify categorical feature indices (LightGBM requires categorical feature indices)
cat_feature_indices = [X_train.columns.get_loc(col) for col in categorical_features]

# Step 5: Initialize and train the LightGBM Regressor
lgb_regressor = lgb.LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42)

# Train the model, specifying the categorical features
lgb_regressor.fit(X_train, y_train, categorical_feature=cat_feature_indices)

# Step 6: Predict and evaluate the model on the test data
y_pred_lgb = lgb_regressor.predict(X_test)
rmse_lgb = np.sqrt(mean_squared_error(y_test, y_pred_lgb))
print(f"LightGBM RMSE without One-Hot Encoding: {rmse_lgb}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004195 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 58
[LightGBM] [Info] Number of data points in the train set: 150826, number of used features: 12
[LightGBM] [Info] Start training from score 43890.785316
LightGBM RMSE without One-Hot Encoding: 68539.90057936781


## Hyperparameter Tuning(Will do it in another notebook to kept improving my lgbm models)

	•	Industry Standard: Select the best-performing models first, then perform hyperparameter tuning on these models.
	•	Why: Tuning is time-consuming, so focusing on the top models ensures efficiency.
	•	Tools: Use Grid Search, Random Search, or Bayesian Optimization for tuning after selecting the best models.

## Ensemble Methods(Will do it in another notebook to kept improving my final models)

# Eveluate and Submit

**LightGBM**

Fisrst Try with RMSE: 73244.92

In [62]:
# Step 1: Define features and target for training
categorical_features = ["brand_model_level", "model_year", "milage", "fuel_type", "transmission", 
                        "ext_col", "int_col", "accident", "clean_title", "horsepower", "engine_size", "cylinders"]
target = "price"

# Step 2: Convert categorical columns to pandas 'category' dtype in both train and test data
for col in categorical_features:
    train_df_final[col] = train_df_final[col].astype('category')
    test_df_final[col] = test_df_final[col].astype('category')

# Step 3: Prepare the training data
X_train_final = train_df_final[categorical_features]
y_train_final = train_df_final[target]

# Step 4: Identify categorical feature indices (LightGBM can handle categorical features natively)
cat_feature_indices = [X_train_final.columns.get_loc(col) for col in categorical_features]

# Step 5: Initialize and train the LightGBM model
lgb_regressor = lgb.LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42)

# Train the model with categorical features
lgb_regressor.fit(X_train_final, y_train_final, categorical_feature=cat_feature_indices)

# Step 6: Prepare the test data for predictions
X_test_final = test_df_final[categorical_features]

# Step 7: Make predictions on the test set
y_test_pred_lgb = lgb_regressor.predict(X_test_final)

# Step 8: Prepare the submission DataFrame
submission_df_lgb = pd.DataFrame({
    'id': test_df['id'],  # Retrieve the 'id' from test_df
    'price': y_test_pred_lgb  # Use the predicted prices
})

# Step 9: Save the submission file
# submission_df_lgb.to_csv('/Users/shudizhao/data_projects/used_car_price_prediction/lgbm_submission_v1.csv', index=False)
submission_df_lgb.to_csv(r'C:\Users\shudi\OneDrive\Desktop\Projects\Kaggle Competition\used_car_price_prediction\lgbm_submission_v1.csv', index=False)

# Check the first few rows of the submission
print(submission_df_lgb.head())

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004874 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 58
[LightGBM] [Info] Number of data points in the train set: 188533, number of used features: 12
[LightGBM] [Info] Start training from score 43878.016178
       id         price
0  188533  19673.600372
1  188534  83148.046702
2  188535  68611.669071
3  188536  33936.971968
4  188537  29530.429970
